In [ ]:
from beartype import beartype
import torch

from edf_interface.data import DemoDataset, TargetPoseDemo, DemoSequence, SE3, PointCloud, DataAbstractBase
from edf_interface.data.utils import units_to_str, str_to_units
from edf_interface.data.preprocess import recursive_apply, rescale, PreprocessDataTypeException, PreprocessNonDataException

In [ ]:
from edf_interface.data.pcd_utils import voxel_filter

In [ ]:
dataset = DemoDataset('../demo/test_demo_single_mug_new')
demo = dataset[0]
pick, place = demo
target_task = place
scene_pcd, grasp_pcd, target_poses = target_task.scene_pcd, target_task.grasp_pcd, target_task.target_poses

In [ ]:
@beartype
@recursive_apply
def downsample(data: DataAbstractBase, voxel_size: float, coord_reduction: str = "average"):
    if type(data) == PointCloud:
        if data.is_empty:
            return data
        else:
            points, colors = voxel_filter(points=data.points, features=data.colors, voxel_size=voxel_size, coord_reduction=coord_reduction)
            return data.new(points=points, colors=colors)
    else:
        if isinstance(data, DataAbstractBase):
            raise PreprocessDataTypeException(f"Unsupported data type: {type(data)}")
        else:
            raise PreprocessNonDataException(f"Unsupported primitive type: {type(data)}")


In [ ]:
demo_downsampled = downsample(demo, voxel_size=0.02)
demo_rescaled = rescale(demo_downsampled, rescale_factor=100.)

In [ ]:
# demo_rescaled[1].show()